In [1]:
from ortools.linear_solver import pywraplp

In [2]:
import pickle
import numpy as np

# Loading the list from the pickle file
file_path = r"F:\dist_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    dist_list = pickle.load(file)

print("Dist list loaded successfully.")
print(np.shape(dist_list))

Dist list loaded successfully.
(400, 10450)


In [3]:
dist_list =  np.transpose(dist_list, (1, 0))

In [15]:
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [60, 120, 50, 100],
    [50, 100, 90, 400],
    [50, 200, 170, 120],
    [50, 200, 190, 160]
]
num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print(f"Worker {i} assigned to task {j}." + f" Cost: {costs[i][j]}")
else:
    print("No solution found.")


Total cost = 240.0

Worker 0 assigned to task 1. Cost: 80
Worker 1 assigned to task 3. Cost: 65
Worker 3 assigned to task 0. Cost: 45
Worker 4 assigned to task 2. Cost: 50


In [4]:
from tqdm import tqdm

costs = dist_list

num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in tqdm(range(num_workers)):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

sol_indexes = []
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.1:
                print(f"Cluster {j} assigned to Class {i}." + f" Cost: {costs[i][j]}")
                sol_indexes.append(i)
else:
    print("No solution found.")

100%|██████████████████████████████████████████████████████████████| 10450/10450 [02:35<00:00, 67.08it/s]


Total cost = 45461.766914785185

Cluster 223 assigned to Class 2. Cost: 110.60854174714295
Cluster 280 assigned to Class 22. Cost: 112.88931929079922
Cluster 220 assigned to Class 52. Cost: 112.60548610422198
Cluster 303 assigned to Class 55. Cost: 136.34389362587925
Cluster 40 assigned to Class 72. Cost: 136.05983022021798
Cluster 245 assigned to Class 89. Cost: 96.55175303741864
Cluster 136 assigned to Class 90. Cost: 81.83749965625245
Cluster 255 assigned to Class 91. Cost: 116.78379091099039
Cluster 338 assigned to Class 92. Cost: 105.7183723644907
Cluster 39 assigned to Class 94. Cost: 141.61846449660072
Cluster 201 assigned to Class 95. Cost: 126.71179989623641
Cluster 244 assigned to Class 103. Cost: 101.08533191318756
Cluster 99 assigned to Class 140. Cost: 111.10479440828348
Cluster 186 assigned to Class 496. Cost: 113.26597793994483
Cluster 150 assigned to Class 497. Cost: 84.50928364822005
Cluster 109 assigned to Class 500. Cost: 111.41191309256752
Cluster 331 assigned to Cl

Cluster 362 assigned to Class 5902. Cost: 130.9151583101278
Cluster 69 assigned to Class 5904. Cost: 100.48498797210704
Cluster 344 assigned to Class 5925. Cost: 105.54856686634446
Cluster 287 assigned to Class 5950. Cost: 107.72326856610273
Cluster 391 assigned to Class 6011. Cost: 119.34864141381989
Cluster 267 assigned to Class 6059. Cost: 102.0486396540615
Cluster 156 assigned to Class 6088. Cost: 130.75960358904746
Cluster 141 assigned to Class 6090. Cost: 94.03610072662008
Cluster 261 assigned to Class 6091. Cost: 120.57724923863448
Cluster 352 assigned to Class 6092. Cost: 73.24602237474286
Cluster 46 assigned to Class 6093. Cost: 101.70454839545638
Cluster 106 assigned to Class 6099. Cost: 73.5595862952276
Cluster 316 assigned to Class 6100. Cost: 135.8762401916732
Cluster 366 assigned to Class 6101. Cost: 163.1703647048704
Cluster 228 assigned to Class 6106. Cost: 62.20216502693598
Cluster 81 assigned to Class 6107. Cost: 68.52488514913053
Cluster 297 assigned to Class 6108. C

Cluster 313 assigned to Class 8446. Cost: 112.80721584187206
Cluster 310 assigned to Class 8447. Cost: 104.7266056948382
Cluster 191 assigned to Class 8450. Cost: 109.50285891454998
Cluster 226 assigned to Class 8497. Cost: 83.73961345205365
Cluster 70 assigned to Class 8529. Cost: 134.86485513689846
Cluster 270 assigned to Class 8546. Cost: 134.88864634392118
Cluster 251 assigned to Class 8549. Cost: 126.38100012909761
Cluster 290 assigned to Class 8552. Cost: 154.05875380332134
Cluster 272 assigned to Class 8554. Cost: 116.21815706743509
Cluster 88 assigned to Class 8566. Cost: 107.87639439360547
Cluster 75 assigned to Class 8568. Cost: 112.12765353583681
Cluster 370 assigned to Class 8902. Cost: 136.09075201670356
Cluster 102 assigned to Class 8904. Cost: 121.87249440022708
Cluster 358 assigned to Class 8925. Cost: 113.5113664864812
Cluster 328 assigned to Class 8950. Cost: 120.02518377189793
Cluster 360 assigned to Class 9011. Cost: 135.9133185034885
Cluster 350 assigned to Class 9

In [7]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"F:\index_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(sol_indexes, file)

print("Combo list saved successfully.")


Combo list saved successfully.
